# Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras, tensorflow
from keras import Sequential, metrics, callbacks, layers, losses
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical, image_dataset_from_directory

# Data

In [2]:
img_size = (256,256)
channels = 3
img_shape = (img_size[0],img_size[1],channels)
batch_size = 64

In [3]:
train_set, val_set = image_dataset_from_directory("/content/drive/MyDrive/Colab Notebooks/Glomerulos/train",
                                        validation_split=0.1,
                                        subset='both',
                                        image_size=img_size,
                                        batch_size=batch_size,
                                        label_mode='categorical',
                                        seed=2023
                                         )
# train_set.batch(batch_size)
# val_set.batch(batch_size)

Found 3544 files belonging to 4 classes.
Using 3190 files for training.
Using 354 files for validation.


In [4]:
data_augmentation = keras.Sequential([
  #layers.Normalization(),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
  layers.RandomBrightness(0.2),
])
data_augmentation.build(img_shape)

# Model

In [8]:
model = keras.Sequential()
# Augmentation
model.add(data_augmentation)

# Backbone
model.add(layers.Conv2D(filters=1,kernel_size=5,strides=(2,2),padding="same",activation="relu",input_shape=img_shape))
model.add(layers.Conv2D(filters=1,kernel_size=3,strides=(2,2),padding="same",activation="relu",input_shape=(img_shape[0]/2,img_shape[1]/2)))
model.add(layers.Conv2D(filters=1,kernel_size=3,strides=(2,2),padding="same",activation="relu",input_shape=(img_shape[0]/4,img_shape[1]/4)))
model.add(layers.Flatten())

# FCs
model.add(layers.Dense(256,activation=('relu'),input_dim=1024))
model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Dropout(.3))
model.add(layers.Dense(64,activation=('relu')))
model.add(layers.Dropout(.2))
model.add(layers.Dense(4,activation=('softmax')))

model.build((batch_size,img_size[0],img_size[1],3))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 3)            0         
                                                                 
 conv2d_3 (Conv2D)           (64, 128, 128, 1)         76        
                                                                 
 conv2d_4 (Conv2D)           (64, 64, 64, 1)           10        
                                                                 
 conv2d_5 (Conv2D)           (64, 32, 32, 1)           10        
                                                                 
 flatten_1 (Flatten)         (64, 1024)                0         
                                                                 
 dense_3 (Dense)             (64, 256)                 262400    
                                                                 
 batch_normalization_1 (Bat  (64, 256)                

# Training

In [9]:
epochs=50
learn_rate=.001
train_metrics = [metrics.CategoricalAccuracy(), metrics.F1Score()]
checkpoint = callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/best_model",
                             monitor='val_loss', verbose=1,
                             save_best_only=True,
                             mode='auto')

early = callbacks.EarlyStopping(monitor='loss', patience=5)

train_callbacks = [callbacks.History(), checkpoint, early]

model.compile(loss=losses.CategoricalCrossentropy(),
              optimizer=Adam(learning_rate=learn_rate),
              metrics=train_metrics
              )

In [ ]:
history=model.fit(
    train_set,
    epochs=epochs,
    shuffle=True,
    validation_data=val_set,
    verbose=1,
    callbacks=train_callbacks,
)